# This notebook is a library
### It contains the followin functions:
1. **generate_data**: generates data for a learning process by means of simulation.

# Imports

In [1]:
import os
import time
import pickle
import random
import numpy as np
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

import keras
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
from keras import Sequential
from keras.layers import Dense, Activation, MaxPooling1D, Dropout, BatchNormalization, Conv1D, Flatten
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2, l1_l2
from keras.metrics import mean_squared_error

from sklearn.model_selection import train_test_split


# Generate Data

In [3]:
def generate_data(N, M, nextConfig, sample):
    """
    This function generates data for a learning process by means of simulation.
    
    The data consists of ITEMS. Each ITEM is a pair: histogram and a configuration 
    (for now, the configuration is scalar / a floating point value).
    
    A histogram consists of sample points. 
    Each sample points is drawn from a distribution, with some specific configuration.
    
    Parameter setting for distribution = Configuration, eg.., alpha, gamma, alpha+LOC
    
    Returns:
        - a matrix of histograms and 
          an array of corresponding configs (one config for each histogram row).

    Arguments:
        - N: number of ITEMS to generate (rows)
        - M: number of sample points that make a histogram (columns)
        - nextConfig: A generator that gives a new alpha/gamma/delta, in each call.
            Example: nextConfig()
                lambda = random(1,10)
                return lambda
        - sample(config, size): returns sample data points from a distribution (size = num samples).
            Example sample(alpha, size=256) 
                ys = yulesimon(alpha, size)
                return ys

    """
    
    items_matrix = np.zeros((N, M), dtype=int)
    config_array = np.zeros((N,), dtype=float)
    
    # generate items_matrix:
    #   repeat N times:
    #     draw M sample points from distribution
    for i in range(N):
        
        # start item
        if callable(nextConfig):
            config = nextConfig()
        else:
            config = nextConfig # in case we pass a numeric config

        # sample data points with current config and add to sample_matrix
        items_matrix[i, :] = sample(config, size=M)
        
        # append corresponding config
        config_array[i] = config
    
    # create histograms from rows of items_matrix
    nbins = np.max(items_matrix)
        
    histogram_matrix = np.apply_along_axis(
        lambda a: np.histogram(a, bins=nbins, range=(1,nbins))[0], 1, items_matrix)

    return items_matrix, histogram_matrix, config_array
        

## Log scale & normalize

In [ ]:
def log_scale(H, C=0):
    # log scale H rows
    # (shift H values by one so as not to take log of zero)
    return np.apply_along_axis(lambda a: np.log10(a + 1 + C), 1, H)

def normalize(H):
    # normalize values (sum to 1)
    return H / H.sum(axis=1, keepdims=1)

# DNN Model

In [ ]:
def create_dnn_model(n_features, 
                 layers, 
                 activation='relu', 
                 init='he_uniform', 
                 batch_normalization=False, 
                 dropout=0, 
                 optimizer='adam', 
                 k_reg=False, 
                 k_reg_lr=0.001, 
                 a_reg=False, 
                 a_reg_lr=0.001):

    model = Sequential()
    
    # ============
    # input-layer
    # ============
    model.add(Dense(units=layers[0]
                      , input_dim=n_features
                      , kernel_initializer=init
                      , kernel_regularizer=l2(k_reg_lr) if k_reg else None
                      , activity_regularizer=l2(a_reg_lr) if a_reg else None
                      , use_bias= (not batch_normalization)
                    ))
    
    
    if batch_normalization:
        model.add(BatchNormalization())
    
    model.add(Activation(activation))

    if dropout > 0:
        model.add(Dropout(dropout))

    # ==============
    # hidden-layers
    # ==============
    for units in layers[1:]:
        model.add(Dense(units=units
                        , kernel_initializer=init
                        , kernel_regularizer=l2(k_reg_lr) if k_reg else None
                        , activity_regularizer=l2(a_reg_lr) if a_reg else None
                        , use_bias= (not batch_normalization)
                        ))

    if batch_normalization:
        model.add(BatchNormalization())

    model.add(Activation(activation))
    
    if dropout > 0:
        model.add(Dropout(dropout))

    # =============
    # output-layer
    # =============
    model.add(Dense(units=1
                    , kernel_initializer=init
                    , kernel_regularizer=l2(k_reg_lr) if k_reg else None
                    , activity_regularizer=l2(a_reg_lr) if a_reg else None
                    , use_bias= (not batch_normalization)
                    ))
    
    if batch_normalization:
        model.add(BatchNormalization())

    model.add(Activation('linear'))

    model.compile(loss='mse', metrics=['mse'], optimizer=optimizer)

    return model

# CNN Model

In [1]:
def create_cnn_model(n_features, filters=32):
    
    model = Sequential()

    model.add(Conv1D(32, 3, activation="relu", input_shape=(n_features, 1)))
    
    model.add(MaxPooling1D(pool_size=2))

    model.add(Flatten())

    model.add(Dense(64, activation="relu"))

    model.add(Dense(1))
    
    model.compile(loss="mse", optimizer="adam")

    return model

# Training

In [ ]:
def train_model(model, X_train, y_train, batch_size=32):

    # split train/val
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

    # early-stopping
    es_patience = 50
    es = EarlyStopping(monitor='val_loss', 
                        patience=es_patience, 
                        mode='min', 
                        restore_best_weights=True, 
                        verbose=0)
    
    # reduce learning-rate on plateau
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=10)
    
    # model checkpoint
    if not os.path.exists('models'):
        os.makedirs('models')
    date_str = datetime.now().strftime('%d-%m-%Y_%H-%M-%S')   
    base_path           = f'models/DNN_{date_str}'
    model_path          = f'{base_path}.h5'
    history_path        = f'{base_path}.history'
    cp = ModelCheckpoint(filepath=model_path, monitor='val_loss', mode='min', save_best_only=True, verbose=0)

    # fit model
    history = model.fit(X_train, 
                        y_train, 
                        validation_data=(X_val, y_val), 
                        epochs=200, 
                        batch_size=batch_size, 
                        shuffle=False, 
                        callbacks=[es, reduce_lr, cp], 
                        verbose=0)
    
    # save history with model
    with open(history_path, 'wb') as f:
        pickle.dump(history.history, f)
    
    # load best weights from last checkpoint
    model = keras.models.load_model(model_path)
    return model, history.history

# Learning Curves

In [ ]:
def plot_learning_curves(history, train_key='loss', val_key='val_loss'):
    plt.figure(figsize=(2,2))
    plt.plot(history[train_key])
    plt.plot(history[val_key])
    plt.title('learning curves')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper right')
    # plt.show()

# Create and train DNN model

In [ ]:
def dnn_trial(X_train, y_train, X_test, y_test):
    dnn_model = create_dnn_model(X_train.shape[1], 
                            layers=[256,256], 
                            activation='relu', 
                            init=keras.initializers.HeUniform(), 
                            batch_normalization=True, 
                            dropout=0.0, 
                            optimizer=Adam(learning_rate=1e-2), 
                            k_reg=True,
                            k_reg_lr=1e-5, 
                            a_reg=True,
                            a_reg_lr=1e-5)

    # train
    dnn_model, history = train_model(dnn_model, X_train, y_train)
    
    # predict
    y_pred = dnn_model.predict(X_test).flatten()

    # mse
    mse = mean_squared_error(y_test, y_pred)

    # sqrt_mse
    sqrt_mse = np.sqrt(mse)

    return dnn_model, history, y_pred, sqrt_mse